In [64]:
import boto3

from boto3.dynamodb.conditions import Key, Attr

import pandas as pd

from pandasgui import show 

In [49]:
dynamodb = boto3.resource( 'dynamodb' )

In [50]:
adspp_table = dynamodb.Table( 'ads_passenger_processed' )

In [51]:
def attr_list( table ):

    scan_input = dict(
                      Limit = 1
                     )

    scan_output = table.scan( **scan_input )

    first_item_of_table = scan_output[ 'Items' ][ 0 ]

    list_of_attrs_of_table = list( first_item_of_table.keys() )

    return list_of_attrs_of_table

In [52]:
adspp_attr_list = attr_list( adspp_table )

In [53]:
print( ', '.join( adspp_attr_list ) )

yawRate, drivingMode, topic, msgsize, time, gearLocation, metadataID, throttlePercentage, steeringPercentage, brakePercentage, engageAdvice, groupMetadataID, errorCode, signal, steeringRate, header, speedMps, engineStarted, _id, fuelRangeM, msg_type, wheelSpeed


In [79]:
query_input = dict(
                   IndexName = 'topic-index',
                   Select = 'SPECIFIC_ATTRIBUTES',
                   # Limit =
                   ConsistentRead = False,
                   ScanIndexForward = True,
                   # ExclusiveStartKey =
                   # ReturnConsumedCapacity =
                   ProjectionExpression = 'yawRate, drivingMode, topic, msgsize, #time, gearLocation, metadataID, ' +
                                          'throttlePercentage, steeringPercentage, brakePercentage, engageAdvice, ' +
                                          'groupMetadataID, errorCode, #signal, steeringRate, header, speedMps, ' + 
                                          'engineStarted, #id, fuelRangeM, msg_type, wheelSpeed',
                   FilterExpression = Key( 'groupMetadataID' ).eq( '1f70a4f0-f0e0-11ee-ba1e-fb353e7798cd' ),
                   KeyConditionExpression = Key( 'topic' ).eq( '/apollo/canbus/chassis' ),
                   ExpressionAttributeNames = {
                                               '#id': '_id',
                                               '#time': 'time',
                                               '#signal': 'signal'
                                              }
                   # ExpressionAttributeValues =
                  )

query_output = adspp_table.query( **query_input )

query_output_df = pd.DataFrame.from_dict( pd.json_normalize( query_output[ 'Items' ] ), orient = 'columns' )

In [82]:
print( 'LastEvaluatedKey' in query_output.keys() )

True


In [ ]:
i = 0
while ( 'LastEvaluatedKey' in query_output.keys() ):

    print( query_output_df.shape )

    query_input[ 'ExclusiveStartKey' ] = query_output[ 'LastEvaluatedKey' ]

    query_output = adspp_table.query( **query_input )

    temp_df = pd.DataFrame.from_dict( pd.json_normalize( query_output[ 'Items' ] ), orient = 'columns' )

    query_output_df = pd.concat( [ query_output_df, temp_df ] )

    i = i + 1

In [1]:
# query_output_df.to_csv( r'/home/rcoldren_linux/Desktop/Data.csv', index = False )

NameError: name 'query_output_df' is not defined